In [ ]:
import pandas as pd
import numpy as np
from pyqubo import Array
import scipy as sp
import threading as t


# T  - max count of cycles
# C  - transport capacity
# N  - count of nodes
# A  - count of transports

T = 15
C = 10
N = 57
A = 15

data_adj = pd.read_csv('task-2-adjacency_matrix.csv').to_numpy()
data_node = pd.read_csv('task-2-nodes.csv', header=None).to_numpy()
names_adj, dist_adj = data_adj[:, 0], data_adj[:, 1:]
names_node, dist_node = data_node[:, 0], data_node[:, 1:]

dict_adj = dict()
dict_node = dict()

i = 0
for elm in data_adj:
    dict_adj.update({i: elm[0]})
    i += 1
    
for elm in data_node:
    dict_node.update({elm[0]: elm[1]})
    
dist_adj = np.asarray(
    [[int(dist_adj[i, j]) if dist_adj[i, j] not in ["-", "0"] else 1e9 
      for j in range(len(dist_adj[0]))] for i in range(len(dist_adj))])

W = np.unique(dist_adj)[-2]

intersections = set()
depot_ind = 0
for i in range(N):
    if dict_adj.get(i).split(' ')[0] == 'Вокзал':
        depot_ind = i
    elif dict_node.get(dict_adj.get(i)) == 0:
        intersections.add(i)
        
x = Array.create('x', shape=(A, N, T), vartype='BINARY')
y = Array.create('y', shape=(A, C), vartype='BINARY')

H_a = sum(sum(sum(sum(dist_adj[v, k]*x[a, v, s]*x[a, k, s+1]/W 
          for s in range(T-1)) 
          for v in range(N) if dist_adj[v, k] != 1e9) 
          for k in range(N)) 
          for a in range(A))

H_b1 = sum((1 - sum(sum(x[a, v, s]
            for s in range(T)) 
            for a in range(A)))**2 
            for v in range(N) if v != depot_ind and v not in intersections)

H_b2 = sum(sum((1 - sum(x[a, v, s]
            for v in range(N)))**2
            for s in range(T))
            for a in range(A))

H_c =  sum((sum(sum(sum(dict_node.get(dict_adj.get(l))*x[a, l, j] 
                        for l in range(N) if l not in intersections and l != depot_ind) 
                        for j in range(b)) 
                        for b in range(1, T)) - sum(y[a, c] for c in range(C)))**2 
                        for a in range(A))

H = H_a + H_b1 + H_b2 + H_c

model = H.compile()

qubo, offset = model.to_qubo()

Q_matrix = np.zeros(shape = (A*N*T + A*C, A*N*T + A*C))

def work(v1b,v1e):
    global Q_matrix, qubo, A, N, T, C
    for a1 in range(A):
        for a2 in range(A):
            for v1 in range(v1b, v1e):
                for v2 in range(N):
                    for s1 in range(T):
                        for s2 in range(T):
                            for c1 in range(C):
                                for c2 in range(C):
                                    try:
                                        Q_matrix[A*N*T + a1*C + c1][A*N*T + a2*C + c2] = qubo[f'y[{a1}][{c1}]', f'y[{a2}][{c2}]']
                                        Q_matrix[A*N*T + a2*C + c2][A*N*T + a1*C + c1] = qubo[f'y[{a2}][{c2}]', f'y[{a1}][{c1}]']
                                        Q_matrix[a1*T*N + v1*T + s1][a2*T*N + v2*T + s2] = qubo[f'x[{a1}][{v1}][{s1}]', f'x[{a2}][{v2}][{s2}]']
                                        Q_matrix[a2*T*N + v2*T + s2][a1*T*N + v1*T + s1] = qubo[f'x[{a2}][{v2}][{s2}]', f'x[{a1}][{v1}][{s1}]']
                                        Q_matrix[a1*T*N + v1*T + s1][A*N*T + a2*C + c2] = qubo[f'x[{a1}][{v1}][{s1}]', f'y[{a2}][{c2}]']
                                        Q_matrix[A*N*T + a2*C + c2][a1*T*N + v1*T + s1] = qubo[f'y[{a2}][{c2}]', f'x[{a1}][{v1}][{s1}]']
                                        Q_matrix[a2*T*N + v2*T + s2][A*N*T + a1*C + c1] = qubo[f'x[{a2}][{v2}][{s2}]', f'y[{a1}][{c1}]']
                                        Q_matrix[A*N*T + a1*C + c1][a2*T*N + v2*T + s2] = qubo[f'y[{a1}][{c1}]', f'x[{a2}][{v2}][{s2}]']
                                    except KeyError:
                                        pass
    

threads = list()
threads.append(t.Thread(target=work, args=[0,5]))
threads.append(t.Thread(target=work, args=[5,10]))
threads.append(t.Thread(target=work, args=[10,15]))
threads.append(t.Thread(target=work, args=[15,20]))
threads.append(t.Thread(target=work, args=[20,25]))
threads.append(t.Thread(target=work, args=[25,30]))
threads.append(t.Thread(target=work, args=[30,35]))
threads.append(t.Thread(target=work, args=[35,40]))
threads.append(t.Thread(target=work, args=[40,45]))
threads.append(t.Thread(target=work, args=[45,50]))
threads.append(t.Thread(target=work, args=[50,57]))

for thread in threads:
    thread.start()
    
print(Q_matrix)
    
sp.sparse.save_npz('qubo.npz', sp.sparse.csr_matrix(Q_matrix))